In [ ]:
#This code detects and analyses bursting and network bursting events of 
#neuronal networks recorded on Cytoview micro-electrode arrays (MEAs) from Axion BioSystems. 
#Recordings were performed using a Maestro Pro MEA system equipped with AxIS Navigator software. 
#The sampling frequency was set at 12.5 kHz. 
#For spike detection, an adaptive threshold of +/- 6 standard deviations of the noise was applied, creating .spk files
#From these .spk files, time-stamps per electrode were obtained using AxionFileLoader toolbox in MATLAB.
#These time-stamps are saved in csv-format called 'Axion_PT_all.csv'


#Preparations: 
#Make a "well_cell_lines.csv" and copy it to every folder where PT_all.csv files are located. Examples can be found online
#This file needs to be comma-separated!
#Save "Well_annotations_axion(_96wp).csv" and "Electrode_annotations_axion_(96wp).csv" to a local directory.


#Internal notes:
#This code is adapted to work on the MEA analysis PC. 
#Next, check in this code that ", fileEncoding="UTF-8-BOM" is included after every path within read.csv(path). 
#Otherwise, junk text (ï..) will appear in the first column name of the data frame. 

In [ ]:
#Import libraries
library(stringr)
library(tidyverse)
library(dplyr)
library(data.table)

In [ ]:
#Please check that the correct (or higher) versions are imported:
#ggplot2: 3.4.2
#tibble:  3.2.1
sessionInfo()

In [ ]:
#Path to main folder containing subfolders with Axion_PT_all
parent.folder<-"insert/folder/name"

In [ ]:
#Search for all files with pattern 'Axion_PT_all' and list directories where these are found
axion_PT_all <- list.files(parent.folder, full.names = T, recursive = T, pattern = "Axion_PT_all")
all_axion_dir<-dirname(axion_PT_all)
all_axion_dir
print(length(all_axion_dir))

In [ ]:
#Select patterns to include (e.g. plate numbers, DIVs, basal) to select files for analysis
plate <- c('118-2257')
Day <- c('DIV35','DIV42', 'DIV49')

all_axion_dir <- all_axion_dir[grepl(paste(plate, collapse = '|'), all_axion_dir) &
                              grepl(paste(Day, collapse = '|'), all_axion_dir)]

#Show new list of directories
print(length(all_axion_dir))
all_axion_dir

In [ ]:
#Electrode burst (EB) and network burst (NB) parameters
min_electrodes_proportion<-0.4 #0.4 #proportion of active electrodes to consider a NB
sigmas<-c(30, 80, 120) #time windows for NB parameters #50 - 100
sigma_selected<-sigmas[3]
local_region_min_nae<-0
#EB parameters
Max_interval_to_start_bursts=0.05 #in seconds #0.05=50#ms #potential burst start two spikes with less than 50 ms 
Max_interval_to_end_a_burst=0.05 #in seconds #0.05=50#ms
Min_interval_between_bursts=0.1 #in seconds #0.1=100#ms
Min_duration_of_burst=0.05 #0.05=50#ms 
Min_number_of_spikes_in_burst=5
#NB parameters
Min_duration_of_NB=0.05 #in seconds
Max_duration_of_NB=10 #in seconds
Min_freq_of_NB=2.5 #in Hz
min_electrodes_proportion_in_Min_freq_of_NB=0.33 #proportion of electrodes with Min_freq_of_NB


#High frequency (hf) NB parameters. 
#First set both of these to 0 and optimize the parameters above. 
#Using NB_hfNB-discrimination.ipynb, the values best used for both of these parameters can be determined. 
hfNB_max_duration=0
hfNB_max_time_interval=0

In [ ]:
#Well and electrode annotations of Axion plates

#Uncomment the two rows below if you used 96-well plates with 8 electrodes
# Well_annotations_axion<-read.csv("insert_path/Well_annotations_axion_96wp.csv")
# Electrode_annotations_axion<-read.csv("insert_path/Electrode_annotations_axion_96wp.csv")

#Uncomment the two rows below if you used 24- or 48-well plates with 16 electrodes
Well_annotations_axion<-read.csv("insert_path/Well_annotations_axion.csv", fileEncoding="UTF-8-BOM")
Electrode_annotations_axion<-read.csv("insert_path/Electrode_annotations_axion.csv", fileEncoding="UTF-8-BOM")

#Define recording time and number of electrodes here
recording_time_sec_axion<-5*60
electrodes_per_well_axion<-16 #8#16, 8 for the 96 wells.  


Functions

Electrode bursts functions

In [ ]:
#Run to create function to find bursts with maximum interval algorithm, modified from meaRtools.
mi_find_bursts <- function(spikes, mi_par) {

  ## For one spike train, find the burst using max interval method.
  no_bursts <- matrix(nrow = 0, ncol = 1) # emtpy matrix nrow()=length() = 0.
  par <- mi_par
  beg_isi <- par$beg_isi
  end_isi <- par$end_isi
  min_ibi <- par$min_ibi
  min_durn <- par$min_durn
  min_spikes <- par$min_spikes

  nspikes <- length(spikes)

  ## Create a temp array for the storage of the bursts.  Assume that
  ## it will not be longer than nspikes divided by 2 since we need at least two
  ## spikes to be in a burst.

  max_bursts <- floor(nspikes / 2)
  bursts <- matrix(NA, nrow = max_bursts, ncol = 3)
  colnames(bursts) <- c("beg", "end", "ibi") #time of initiation, end, and inter burst interval
  burst <- 0 # current burst number


    
  ## Phase 1 -- burst detection.  Here a burst is defined as starting
  ## when two consecutive spikes have an ISI less than BEG.ISI apart.
  ## The end of the burst is given when two spikes have an ISI greater
  ## than END.ISI.
  ## Find ISIs closer than beg_isi, and end with end isi.
  ## LAST.END is the time of the last spike in the previous burst.
  ## This is used to calculate the IBI.
  ## For the first burst, this is no previous IBI
  last_end <- NA; # for first burst, there is no IBI.

  n <- 2
  in_burst <- FALSE

  while (n <= nspikes) {

    next_isi <- spikes[n] - spikes[n - 1]
    if (in_burst) {
      if (next_isi > end_isi) {
        ## end of burst
        end <- n - 1; in_burst <- FALSE


        ibi <- spikes[beg] - last_end; last_end <- spikes[end]
        res <- c(beg, end, ibi)
        burst <- burst + 1
        if (burst > max_bursts) {
          
          browser()
        }
        bursts[burst, ] <- res
      }
    } else {
      ## not yet in burst.
      if (next_isi < beg_isi) {
        ## Found the start of a new burst.
        beg <- n - 1; in_burst <- TRUE
      }
    }
    n <- n + 1
  }

  ## At the end of the burst, check if we were in a burst when the
  ## train finished.
  if (in_burst) {
    end <- nspikes
    ibi <- spikes[beg] - last_end
    res <- c(beg, end, ibi)
    burst <- burst + 1
    if (burst > max_bursts) {
     
      browser()
    }
    bursts[burst, ] <- res
  }

  ## Check if any bursts were found.
  if (burst > 0) {
    ## truncate to right length, as bursts will typically be very long.
    bursts <- bursts[1:burst, , drop = FALSE]
  } else {
    ## no bursts were found, so return an empty structure.
    return(no_bursts)
  }


    
    
  ## Phase 2 -- merging of bursts.  Here we see if any pair of bursts
  ## have an IBI less than MIN.IBI; if so, we then merge the bursts. 
  ibis <- bursts[, "ibi"]
  merge_bursts <- which(ibis < min_ibi)

  if (any(merge_bursts)) {
    ## Merge bursts efficiently.  Work backwards through the list, and
    ## then delete the merged lines afterwards.  This works when we
    ## have say 3+ consecutive bursts that merge into one.

    for (burst in rev(merge_bursts)) {
      bursts[burst - 1, "end"] <- bursts[burst, "end"]
      bursts[burst, "end"] <- NA 
    }
    bursts <- bursts[- merge_bursts, , drop = FALSE] # delete the unwanted info.
  }

    
    
    
    
  ## Phase 3 -- remove small bursts: less than min duration (MIN.DURN), or
  ## having too few spikes (less than MIN.SPIKES).
  ## In this phase we have the possibility of deleting all spikes.
  ## LENis the number of spikes in a burst.
  ## DURN is the duration of burst.
  len <- bursts[, "end"] - bursts[, "beg"] + 1
  durn <- spikes[bursts[, "end"]] - spikes[bursts[, "beg"]]
  bursts <- cbind(bursts, len, durn)

  rejects <- which(
    (durn < min_durn) | (len < min_spikes))

  if (any(rejects)) {
    bursts <- bursts[- rejects, , drop = FALSE]
  }

  if (nrow(bursts) == 0) {
    ## All the bursts were removed during phase 3.
    bursts <- no_bursts
  } else {
    ## Compute mean ISIs
    len <- bursts[, "end"] - bursts[, "beg"] + 1
    durn <- spikes[bursts[, "end"]] - spikes[bursts[, "beg"]]
    mean_isis <- durn / (len - 1)

    ## Recompute IBI (only needed if phase 3 deleted some cells).
    if (nrow(bursts) > 1) {
      ibi2 <- c(NA, .calc_ibi(spikes, bursts))
    } else {
      ibi2 <- NA
    }
    bursts[, "ibi"] <- ibi2

    si <- rep(1, length(mean_isis))
    bursts <- cbind(bursts, mean_isis, si)
  }

  ## End -- return burst structure.
  bursts

}





#meaRtools function to summarize burst information
get_burst_info <- function(allb, index) {
  ## Extra some part of the Burst information, for each channel.
  ## index will be the name of one of the columns of burst info.
  ## This is a HELPER function for calc_burst_summary
  sapply(allb, function(b) {
    if (length(b) > 1) {
      b[, index]
    } else {
      0
    }
  }
  , simplify = FALSE)
}

calc_burst_summary <- function(s, bursty_threshold=1) {#bursty_threshold was 1
  ## bursty_threshold: min number of  bursts per minute to count as
  ## a bursty unit.

  ## Compute the summary burst information.
  ## The columns of the data.frame returned.
  ## channels: electrode name
  ## spikes: #spikes
  ## mean_freq: firing rate (Hz)
  ## nbursts: #bursts detected
  ## bursts_per_sec: #bursts per second.matrix, nrow is 0,ncol is 1
  ## bursts_per_min: #bursts per min
  ## bursty: is bursts_per_min bigger than bursty_threshold
  ## (defaults to 1 burst per min)
  ## mean_dur: mean burst duration
  ## sd_dur: sd
  ## mean_spikes : mean #spikes in a burst
  ## sd_spikes: sd
  ## per_spikes_in_burst : % of spikes in a burst
  ## per_spikes_out_burst: % of spikes not in a burst
  ## mean_si: mean Surprise Index (only for poisson .surprise measure)
  ## mean_isis: mean ISI within a burst (old name: mean2.isis)
  ## sd_mean_isis: sd
  ## mean_ibis: mean IBI
  ## sd_ibis: sd
  ## cv_ibis: Coefficient of variation of IBI (equals mean_ibi
  ## divided by sd_ibi)

  allb <- s$allb

  ## Create a table of output results.
  channels <- s$channels
  spikes <- as.vector(s$nspikes)
  duration <- recording_time_sec
  mean_freq <- round(spikes / duration, 3)
  nbursts <- sapply(allb, .num_bursts)
  bursts_per_sec <- round(nbursts / duration, 3)
  bursts_per_min <- bursts_per_sec * 60
  bursty <- ifelse(bursts_per_min >= bursty_threshold, 1, 0)
  durations <- get_burst_info(allb, "durn")
  mean_dur <- round(sapply(durations, mean), 3)
  sd_dur <- round(sapply(durations, sd), 3)
  isis <- .calc_all_isi(s, allb)
  mean_isis <- sapply(isis, mean)
  sd_isis <- unlist(sapply(isis, sd, na.rm = TRUE))
  ns <- get_burst_info(allb, "len")
  mean_spikes <- round(sapply(ns, mean), 3)
  sd_spikes <- round(sapply(ns, sd), 3)
  total_spikes_in_burst <- sapply(ns, sum)
  per_spikes_in_burst <- round(100 * (total_spikes_in_burst / spikes), 3)
  si <- get_burst_info(allb, "si")
  mean_si <- round(sapply(si, mean), 3)
  ibis <- .calc_all_ibi(s, allb)
  mean_ibis <- sapply(ibis, mean)
  sd_ibis <- sapply(ibis, sd, na.rm = TRUE)
  cv_ibis <- round(sd_ibis / mean_ibis, 3)
  mean_ibis <- round(mean_ibis, 3); sd_ibis <- round(sd_ibis, 3)

  df <- data.frame(channels = channels, spikes = spikes, mean_freq = mean_freq,
    nbursts = nbursts,
    bursts_per_sec = bursts_per_sec,
    bursts_per_min = bursts_per_min,
    bursty = bursty,
    mean_dur = mean_dur,
    sd_dur = sd_dur,
    mean_spikes = mean_spikes,
    sd_spikes = sd_spikes,
    per_spikes_in_burst = per_spikes_in_burst,
    per_spikes_out_burst = round(100.0 - per_spikes_in_burst, 3),
    mean_si = mean_si,
    mean_isis = mean_isis,
    sd_mean_isis = sd_isis,
    mean_ibis = mean_ibis,
    sd_ibis = sd_ibis,
    cv_ibis = cv_ibis
  )
  df
}

.plot_burst_info <- function(allb, index, ylab=NULL, max=- 1, title="") {
  ## Plot result of burst analysis in a stripchart, one col per channel.
  ## Feature plotted is given by index, e.g. "durn", "len".
  plot_channels <- length(allb) # plot all channels.

  values <- list()
  for (i in 1:plot_channels) {
    b <- allb[[i]]
    if (.num_bursts(b) == 0) {
      res <- NULL
    } else {
      res <- b[, index]
    }

    infs <- which(res == Inf)
    if (length(infs) > 0)
    res <- res[- infs]

    values[[i]] <- res
  }

  if (max > 0) {
    values <- sapply(values, pmin, max)
  }
  mins <- min(sapply(values, min), na.rm = TRUE)
  maxs <- max(sapply(values, max), na.rm = TRUE)

  if (is.null(ylab))
    ylab <- index

  stripchart(values, method = "jitter", pch = 20, vert = TRUE, main = title,
    ylim = c(mins, maxs),
    xlab = "channel", ylab = ylab)

}

.num_bursts <- function(b) {
  ## Return the number of bursts found for a spike train.
  if (is.na(b[1]))
    0
    else
  nrow(b)
}

.calc_all_isi <- function(s, allb) {
  ## Compute ISI within bursts for all spike trains.

  calc_isi <- function(spikes, b) {
    ## for one spike train, get all ISIs within bursts in that train.
    if (.num_bursts(b) == 0) {
      return(NA)
    }

    ## as.vector is needed below in case each burst is of the same
    ## length (in which case an array is returned by apply).  In
    ## normal cases, bursts are of different lengths, so "apply"
    ## returns a list.

    isis <- as.vector(
      unlist(apply(b, 1,
        function(x) {
          diff(spikes[x[1]:x[2]])
        })))
  }

  nchannels <- s$NCells
  isis <- list()
  for (i in 1:nchannels) {
    isis[[i]] <- calc_isi(s$spikes[[i]], allb[[i]])
  }

  isis
}

.calc_all_ibi <- function(s, allb) {
  ## Compute IBI for all spike trains.
  nchannels <- s$NCells
  ibis <- list()
  for (i in 1:nchannels) {
    ibis[[i]] <- .calc_ibi(s$spikes[[i]], allb[[i]])
  }

  ibis
}

.calc_ibi <- function(spikes, b) {
  ## Compute the interburst intervals (IBI) for one spike train.
  ## Only valid if more than one burst.

  nburst <- .num_bursts(b)
  if (nburst == 0) {
    res <- NA # no bursts
  } else {
    if (nburst == 1) {
      res <- NA # cannot compute  IBI w/only 1 burst.
    } else {
      ## find end spike in each burst.
      end <- b[, "beg"] + b[, "len"] - 1

      ## for n bursts, there will be n bursts minus 1 IBIs.
      start_spikes <- b[2:nburst, "beg"]
      end_spikes <- end[1:(nburst - 1)]
      res <- spikes[start_spikes] - spikes[end_spikes]
    }
  }
  res
}

.mean_burst_summary <- function(allb_sum) {
  ## Summarise the burst information.  This does not handle per_spikes_in_burst
  subset <- allb_sum[which(allb_sum$bursty == 1), ]

  fields <- c("spikes", "mean_dur", "cv_ibis", "bursts_per_min",
             "per_spikes_in_burst")
  res <- rep(0, length(fields) * 2)
  names(res) <- paste(rep(fields, each = 2), c("m", "sd"), sep = ".")
  n <- 1
  for (field in fields) {
    dat <- subset[[field]]
    if (length(dat) > 0) {
      mean <- mean(dat, na.rm = TRUE); sd <- sd(dat, na.rm = TRUE);
    } else {
      mean <- sd <- NA;
    }
    res[n] <- mean; res[n + 1] <- sd
    n <- n + 2
  }

  res

}

print("Electrode bursts functions done!")

NB functions

In [ ]:
#Run to create functions to detect and analyse network bursts. Modified functions from meaRtools
.graythresh <- function(I) {
  I <- as.vector(I)
  if (min(I) < 0 | max(I) > 1) {
    stop("Data needs to be between 0 and 1")
  }
  I <- I * 256

  num_bins <- 256 
  counts <- hist(I, num_bins, plot = FALSE)$counts

  # Variables names are chosen to be similar to the formulas in the Otsu paper.
  p <- counts / sum(counts)
  omega <- cumsum(p)
  mu <- cumsum(p * (1:num_bins))
  mu_t <- mu[length(mu)]

  sigma_b_squared <- (mu_t * omega - mu) ^ 2 / (omega * (1 - omega))
  sigma_b_squared[is.na(sigma_b_squared)] <- 0
  maxval <- max(sigma_b_squared)
  isfinite_maxval <- is.finite(maxval)
  if (isfinite_maxval) {
    idx <- mean(which(sigma_b_squared == maxval))
    # Normalize the threshold to the range [0, 1].
    level <- (idx - 1) / (num_bins - 1)
  } else {
    level <- 0.0
  }
  level
}


# Convert the spike list to a matrix, to be consistent with matlab
.nb_prepare <- function(s) {
  spikes <- s$spikes
  colnames <- names(spikes)

  slength <- sapply(spikes, length)
  max_elements <- max(slength)

  data <- matrix(0, length(colnames), max_elements)
  for (i in  1:length(colnames)) {
    data[i, 1:slength[[i]]] <- spikes[[i]]
  }
  rownames(data) <- colnames
  data <- t(data)
  data
}

# Generate binned data for the well
.nb_select_and_bin <- function(data, well, sbegin, send, bin) {
  well_data <- data[, grep(paste0(well,"_"), colnames(data))]
  well_data[well_data < sbegin | well_data > send] <- 0
  if (is.null(dim(well_data))) {
    dim(well_data) <- c(length(well_data), 1)
  }
  if (length(well_data) > 0) {
    well_data_postive <- well_data[well_data > 0]
    if (all(is.na(well_data_postive))) {
      to_add_back <- 0
    } else {
      to_add_back <- min(well_data[well_data > 0])
    }
  } else {
    to_add_back <- 0
  }


  well_data <- well_data[rowSums(well_data) > 0, ]
  well_data <- round(well_data * 12500)
  temp <- well_data[well_data > 0]
  if (length(temp) > 0) {
    t_min <- min(temp)
    t_max <- max(temp)
    well_data <- well_data - t_min + 1
    range <- (t_max - t_min + 1)
    bins <- floor(range / bin)
    if (range %% bin > 0) {
      bins <- bins + 1
    }
    x <- (0:bins) * bin + 1
    y <- c(- Inf, x, max(x) + bin, + Inf)

    if (is.null(dim(well_data))) {
      dim(well_data) <- c(length(well_data), 1)
    }
    well_data_new <- matrix(0, bins + 1, dim(well_data)[2])
    for (i in 1: dim(well_data)[2]) {
      temp <- hist(well_data[, i], breaks = y, plot = FALSE)$counts
      temp <- temp[2:(length(temp) - 1)]
      well_data_new[, i] <- temp
    }
    well_data_new[well_data_new > 1] <- 1
  } else {
    well_data_new <- temp
  }

  list(well_data_new, to_add_back)
}

.nb_gaussian_filter <- function(sigma, well_data, min_electrodes) {
  sigma_input <- sigma
  if (length(well_data) == 0) {
    f2 <- numeric()
  } else {
    if (sigma %% 2 == 0) {
      sigma <- sigma + 1
    }
    filter_size <- sigma
    half_size <- (filter_size - 1) / 2
    x <- seq(- half_size, half_size, length = filter_size)
    gauss_filter <- exp(- x ^ 2 / (2 * sigma ^ 2))
    gauss_filter <- gauss_filter / sum(gauss_filter)
    f <- well_data
    if (length(which(apply(well_data, 2, max) > 0)) >= min_electrodes) {
      for (i in 1:dim(f)[2]) {
        if (max(well_data[, i]) > 0) {
          f[, i] <- stats::filter(well_data[, i], gauss_filter, circular = TRUE)
          f[, i] <- f[, i] / max(f[, i])
        }
      }
      f1 <- rowSums(f)
      f1 <- f1 / max(f1)
      f2 <- stats::filter(f1, gauss_filter, circular = TRUE)
      f2 <- f2 / max(f2)
      dim(f2) <- c(length(f2), 1)
      colnames(f2) <- sigma_input
    } else {
      f2 <- numeric()
    }
  }
  f2
}
##second part
.nb_get_spike_stats <- function(well_data, timespan) {
  stat <- matrix(0, 1, 3)
  if (length(well_data) > 0) {
    # number of active electrodes
    stat[1] <- length(which(apply(well_data, 2, max) > 0))
    # well level mfr
    stat[2] <- sum(well_data) / timespan
    # mfr by active electrodes
    if (stat[1] > 0) {
      stat[3] <- stat[2] / stat[1]
    }
  }
  colnames(stat) <- c("n_active_electrodes",
                      "mfr_per_well", "mfr_per_active_electode")
  stat
}
.nb_get_burst_stats_intensities <-
  function(well_data, f, timespan, bin_time, min_electrodes,min_electrodes_proportion_in_Min_freq_of_NB_numb,
  sbegin, send, offset2=0, bin_size=0.002,hfNB_max_duration,hfNB_max_time_interval) {
  # bin_size is set to 0.002 based on sample rate of 12500/s
 #str(f)
 n <- length(f)
 stat <- matrix(NA, 11, n)
  
  rownames(stat) <- c("mean_NB_time_per_sec",
    "total_spikes_in_all_NBs",
    "percent_of_spikes_in_NBs",
    "spike_intensity",
    "spike_intensity_by_aEs",
    "total_spikes_in_all_NBs_d_nNB",
    "total_spikes_in_all_NBs_d_nNB_d_nae",
    "mean_spikes_in_NB_d_nae",
    "mean_spikes_in_NB_d_nae_d_NB_duration",
    "mean_spikes_in_NB",
    "total_number_of_NBs")
  colnames(stat) <- rep("NA", n)
  
  for (current in 1:n) {
    if (length(f[[current]] > 0)) {
      colnames(stat)[current] <- colnames(f[[current]]) 
    }
  }
  hf_stat<-stat
  rownames(hf_stat) <- c("mean_hf_NB_time_per_sec",
    "total_spikes_in_all_hf_NBs",
    "percent_of_spikes_in_hf_NBs",
    "spike_intensity",
    "spike_intensity_by_aEs",
    "total_spikes_in_all_hf_NBs_d_nNB",
    "total_spikes_in_all_hf_NBs_d_nNB_d_nae",
    "mean_spikes_in_hf_NB_d_nae",
    "mean_spikes_in_hf_NB_d_nae_d_NB_duration",
    "mean_spikes_in_hf_NB",
    "total_number_of_hf_NBs") 
  nb_times <- list()
  hf_nb_times <- list()
  length(nb_times) <- length(f)
  names(nb_times) <- names(f);
      
  length(hf_nb_times) <- length(f)
  names(hf_nb_times) <- names(f);

  for (current  in 1:n) {
    temp <- f[[current]]
    if (length(temp) > 0) {
      n_active_electrodes <- length(which(apply(well_data, 2, max) > 0))
      temp[temp < 0] <- 0
      level <- .graythresh(temp) 
      indicator0 <- temp >= level
    
       indicator <- c(0, indicator0, 0) # prepare for finding intervals
       diff_indicator <- diff(indicator)
       burst_start <- which(diff_indicator == 1)
       burst_end <- which(diff_indicator == - 1)
       if (length(burst_start) != length(burst_end)) {
          stop("Burst region no match")
        }
       exclude_weird<-c()
       for(region in 1:length(burst_start)){
         if(burst_end[region]-burst_start[region]<2){
         exclude_weird<-c(exclude_weird,region)
         indicator0[burst_start[region]:(burst_end[region] - 1)] <- 0
          }
      }
      exclude_weird<-as.numeric(exclude_weird)
        
      if(length(exclude_weird)>0){
      burst_start<-burst_start[-exclude_weird]
      burst_end<-burst_end[-exclude_weird]
     }

     exclude_no_min_e<-c()
        
       if(length(burst_start)>0){#>0
          for (region in 1: length(burst_start)) {
          current_region_nae <-
          length(which(colSums(well_data[
              (burst_start[region]):(burst_end[region] - 1), ]) > 0))
              
            if (current_region_nae < min_electrodes) {
               
                exclude_no_min_e<-c(exclude_no_min_e,region)
               indicator0[burst_start[region]:(burst_end[region] - 1)] <- 0
               
                }
            
             }}
      
      exclude_no_min_e<-as.numeric(exclude_no_min_e)
        
      if(length(exclude_no_min_e)>0){
      burst_start<-burst_start[-exclude_no_min_e]
      burst_end<-burst_end[-exclude_no_min_e]
     }
        
      exclude_short<-c()
      burst_end_ch<-burst_end * 0.002 + offset2
      burst_start_ch<-burst_start * 0.002 + offset2
      durations_vector<-burst_end_ch-burst_start_ch
      if(length(burst_start>0)) { 
      for(my_dur in 1:length(durations_vector)) {
          specific_dur<-durations_vector[my_dur]
          if(specific_dur<Min_duration_of_NB | specific_dur>Max_duration_of_NB){ 
             exclude_short<-c(exclude_short,my_dur)
             indicator0[burst_start[my_dur]:(burst_end[my_dur] - 1)] <- 0
          }
      }
      
      exclude_short<-as.numeric(exclude_short)
     if(length(exclude_short)>0){
      burst_start<-burst_start[-exclude_short]
      burst_end<-burst_end[-exclude_short]
     }}
      

         
     exclude_no_min_freq_in_each_electrode_inNB<-c()
     burst_end_ch<-burst_end * 0.002 + offset2
     burst_start_ch<-burst_start * 0.002 + offset2
     durations_vector<-burst_end_ch-burst_start_ch#play with this   
       if(length(burst_start)>0){#>0
          for (region in 1: length(burst_start)) {
              nb_dur<-durations_vector[region]
              current_region_nae_min_spikes <-
              length(which(colSums(well_data[
                  (burst_start[region]):(burst_end[region] - 1), ])/nb_dur > Min_freq_of_NB))

                if (current_region_nae_min_spikes < min_electrodes_proportion_in_Min_freq_of_NB_numb) {

                    exclude_no_min_freq_in_each_electrode_inNB<-c(exclude_no_min_freq_in_each_electrode_inNB,region)
                   indicator0[burst_start[region]:(burst_end[region] - 1)] <- 0

                }
            
             }}

    exclude_no_min_freq_in_each_electrode_inNB<-as.numeric(exclude_no_min_freq_in_each_electrode_inNB)

    if(length(exclude_no_min_freq_in_each_electrode_inNB)>0){
    burst_start<-burst_start[-exclude_no_min_freq_in_each_electrode_inNB]
    burst_end<-burst_end[-exclude_no_min_freq_in_each_electrode_inNB]
     }
        

    
      hf_indicator0<-indicator0#copy indicator0 and then keep here the hf but remove the "normal"  
      save_hf_NB<-c() 
      burst_end_ch<-burst_end * 0.002 + offset2
      burst_start_ch<-burst_start * 0.002 + offset2
      durations_vector<-burst_end_ch-burst_start_ch#
      hf_burst_start<-c()
      hf_burst_end<-c()
      if(length(burst_start)>0) {
          df_NB_time_intervals<-data.frame("start_t"=burst_start_ch,
                                      "end_t"=burst_end_ch)
        
          df_NB_time_intervals$NB_time_interval_back<-NA
          df_NB_time_intervals$NB_time_interval_forward<-NA
          df_NB_time_intervals$short<-NA
          
          
      
           if(length(burst_start)>=2) {
             
              for (current_NB in 2:length(burst_start)){
                   df_NB_time_intervals[current_NB,"NB_time_interval_back"]<-df_NB_time_intervals[current_NB,"start_t"]-df_NB_time_intervals[current_NB-1,"end_t"]
                  }
              for (current_NB in 1:length(burst_start)-1){
                 df_NB_time_intervals[current_NB,"NB_time_interval_forward"]<-df_NB_time_intervals[current_NB+1,"start_t"]-df_NB_time_intervals[current_NB,"end_t"]
                  }

              df_NB_time_intervals$short<-pmin(df_NB_time_intervals$NB_time_interval_back, df_NB_time_intervals$NB_time_interval_forward,na.rm = TRUE)
              short_NB_time_intervals<-df_NB_time_intervals$short
              for(my_dur in 1:length(durations_vector)) {
                    specific_dur<-durations_vector[my_dur]
                    short_NB_time_interval<-short_NB_time_intervals[my_dur]
                    if(specific_dur<=hfNB_max_duration & short_NB_time_interval<=hfNB_max_time_interval){
                                
                        save_hf_NB<-c(save_hf_NB,my_dur)

                        indicator0[burst_start[my_dur]:(burst_end[my_dur] - 1)] <- 0#exclude de hf from normal NB

                    }else{
                        hf_indicator0[burst_start[my_dur]:(burst_end[my_dur] - 1)] <- 0  
                        }
                    }
                   }
            save_hf_NB<-as.numeric(save_hf_NB)

            if(length(save_hf_NB)>0){
              hf_burst_start<-burst_start[save_hf_NB]#saving hfNB 
              hf_burst_end<-burst_end[save_hf_NB]#saving hfNB 

              burst_start<-burst_start[-save_hf_NB]#eliminating hfNB from normal NB
              burst_end<-burst_end[-save_hf_NB]
                                     }#eliminating hfNB from normal NB                
        }
   
            active_electrodes_per_nb<-c()
  
        #considering frequency in each electrode to calculate mean
        mean_firing_rate_in_nb<-c() 
        burst_start_region_ch<-burst_start * 0.002 + offset2
        burst_end_region_ch<-burst_end * 0.002 + offset2
        durations_vector<-burst_end_ch-burst_start_ch
        if(length(burst_start)>0) {
          for(region in 1:length(burst_start)){
                nb_dur<-durations_vector[region]
                mean_firing_rate<-mean(colSums(well_data[
                  (burst_start[region]):(burst_end[region] - 1), ])/nb_dur)
              
                mean_firing_rate_in_nb<-c(mean_firing_rate_in_nb,mean_firing_rate)
                }}else{mean_firing_rate_in_nb<-c(rep(NA, length(burst_start)))}
        
      
      nb_times_temp <- cbind.data.frame(burst_start, burst_end)
      
      if(length(hf_burst_start)>0){hf_nb_times_temp <- cbind.data.frame(hf_burst_start, hf_burst_end)}#else{print("no hf_NB")}
     
      nb_times_temp$start_t <- burst_start * 0.002 + offset2
      nb_times_temp$end_t <- burst_end * 0.002 + offset2
      
      nb_times_temp$mean_firing_rate_in_nb<-mean_firing_rate_in_nb
      
      nb_times[[current]] <- nb_times_temp
        
        
        #Considering frequency in each electrode to calculate mean
        mean_firing_rate_in_hf_nb<-c() 
        hf_burst_start_region_ch<-hf_burst_start * 0.002 + offset2
        hf_burst_end_region_ch<-hf_burst_end * 0.002 + offset2
        hf_durations_vector<-hf_burst_end_region_ch-hf_burst_start_region_ch
        if(length(hf_burst_start)>0) {
          for(region in 1:length(hf_burst_start)){
                nb_dur<-hf_durations_vector[region]
                mean_firing_rate<-mean(colSums(well_data[
                  (hf_burst_start[region]):(hf_burst_end[region] - 1), ])/nb_dur)
              
                mean_firing_rate_in_hf_nb<-c(mean_firing_rate_in_hf_nb,mean_firing_rate)
                }}else{mean_firing_rate_in_hf_nb<-c(rep(NA, length(hf_burst_start)))}
        
        
      if(length(hf_burst_start)>0){   
      hf_nb_times_temp$hf_start_t <- hf_burst_start * 0.002 + offset2
      hf_nb_times_temp$hf_end_t <- hf_burst_end * 0.002 + offset2

      hf_nb_times_temp$mean_firing_rate_in_hf_nb<-mean_firing_rate_in_hf_nb
          
     hf_nb_times[[current]] <- hf_nb_times_temp}
        
      stat[1, current] <- sum(indicator0) * bin_time / timespan
        
      totalspikes <- rowSums(well_data)
      spikes_in_network_bursts <- sum(totalspikes * indicator0)
      
      #total_spikes_in_all_NBs
      stat[2, current] <- spikes_in_network_bursts
      
      #percent_of_spikes_in_NBs
      stat[3, current] <- stat[2, current] / sum(totalspikes)
        
      total_burst_regions <- sum(burst_end - burst_start)
      
      #spike_intensity
      stat[4, current] <- spikes_in_network_bursts /
                          total_burst_regions # overall Spike Intensity  
      
      stat[5, current] <- stat[4, current] / n_active_electrodes
      
        
      stat[6, current] <- spikes_in_network_bursts / length(burst_start)
      
      
      stat[7, current] <- stat[6, current] / n_active_electrodes
      

      burst_region_length <- burst_end - burst_start
      
        
      total_spikes_per_active_electrode <- totalspikes / n_active_electrodes
      
      spikes_in_region_per_active_electrode <- matrix(0, length(burst_start), 2)
      

          if(length(burst_start)>0){ 
          for (region in 1: length(burst_start)) {
            spikes_in_region_per_active_electrode[region, 1] <-
              sum(total_spikes_per_active_electrode[
                 burst_start[region]:(burst_end[region] - 1)])
            # normalized to Hz per active electrode within burst region
              spikes_in_region_per_active_electrode[region, 2] <-
              spikes_in_region_per_active_electrode[region, 1] /
                  (burst_region_length[region] * bin_time)
              stat[8, current] <- mean(spikes_in_region_per_active_electrode[, 1])
              stat[9, current] <- mean(spikes_in_region_per_active_electrode[, 2])
          }}else{
             stat[8, current] <-0
             stat[9, current] <- 0
             }
      stat[10, current] <- stat[8, current] * n_active_electrodes
      stat[11, current] <- length(burst_start)
              
        
      if(length(hf_burst_start)>0){ 
        #hf_burst_region_length <- hf_burst_end - hf_burst_start
        hf_spikes_in_region_per_active_electrode <- matrix(0, length(hf_burst_start), 2) 
        #"mean_hf_NB_time_per_sec"
        hf_stat[1, current] <- sum(hf_indicator0) * bin_time / timespan
        #"total_spikes_in_all_hf_NBs"
        hf_spikes_in_network_bursts <- sum(totalspikes * hf_indicator0)
        hf_stat[2, current] <- hf_spikes_in_network_bursts
        #"percent_of_spikes_in_hf_NBs"
        hf_stat[3, current] <- hf_stat[2, current] / sum(totalspikes)
        hf_total_burst_regions <- sum(hf_burst_end - hf_burst_start)
        #"spike_intensity",
        hf_stat[4, current] <- hf_spikes_in_network_bursts /
                                  hf_total_burst_regions # overall Spike Intensity
        #"spike_intensity_by_aEs",
        hf_stat[5, current] <- hf_stat[4, current] / n_active_electrodes
        #"total_spikes_in_all_hf_NBs_d_nNB",
        hf_stat[6, current] <- hf_spikes_in_network_bursts / length(hf_burst_start)
        #"total_spikes_in_all_hf_NBs_d_nNB_d_nae",
        hf_stat[7, current] <- hf_stat[6, current] / n_active_electrodes
        hf_burst_region_length <- hf_burst_end - hf_burst_start
        for (region in 1: length(hf_burst_start)) {
        hf_spikes_in_region_per_active_electrode[, 1] <-  #NA
                 sum(total_spikes_per_active_electrode[
                  hf_burst_start[region]:(hf_burst_end[region] - 1)])
                # normalized to Hz per active electrode within burst region
        hf_spikes_in_region_per_active_electrode[, 2] <-#NA
                  hf_spikes_in_region_per_active_electrode[region, 1] /
                      (hf_burst_region_length[region] * bin_time)
        #     "mean_spikes_in_hf_NB_d_nae",
        hf_stat[8, current] <- mean(hf_spikes_in_region_per_active_electrode[, 1])
          #"mean_spikes_in_hf_NB_d_nae_d_NB_duration",
        hf_stat[9, current] <- mean(hf_spikes_in_region_per_active_electrode[, 2])
          # "mean_spikes_in_hf_NB",
        hf_stat[10, current] <- hf_stat[8, current] * n_active_electrodes
        hf_stat[11, current] <- length(hf_burst_start)  
         }
        }else{}
               
      
    }
  }
  list(stat, nb_times,hf_stat, hf_nb_times) 

}

print("NB functions done!")


END NB functions

AXION

In [ ]:
#EB and NB detection/analysing functions are created
#Now apply to Axion_PT_all files and create a subfolder containing csv's with analyses as well as raster plots per well
if(length(all_axion_dir)!=0){
    sf=12500 #sampling frequency
    print("analysing Axion_PT_all")
    #define parameters
    Well_annotations<-Well_annotations_axion
    Electrode_annotations<-Electrode_annotations_axion
    recording_time_sec<-recording_time_sec_axion
    electrodes_per_well<-electrodes_per_well_axion
    min_electrodes<-electrodes_per_well*min_electrodes_proportion 
    min_electrodes_proportion_in_Min_freq_of_NB_numb<-electrodes_per_well*min_electrodes_proportion_in_Min_freq_of_NB
    for(PT_all_dir in all_axion_dir){
        setwd(PT_all_dir)
        batch<-basename(PT_all_dir) 
        DIV_range<-basename(dirname(dirname(PT_all_dir)))
        file<-PT_all_dir
        PT_path<-list.files(full.names = F, recursive = T, pattern = "*Axion_PT_all*")
        PT_all<-read.table(PT_path,sep=",",header = TRUE) #output from PT_all_from_axion
        
        PT_all$Channel_Label<-as.character(PT_all$Channel_Label)
        
        PT_all$Well_Label_num<-paste0("ID_",as.character(PT_all$Well_Label))
        
        PT_all$Well_Label<-Well_annotations$Well_Label[match(PT_all$Well_Label_num,Well_annotations$Well_Label_num)]
        PT_all$Well_Label<-factor(PT_all$Well_Label, levels=unique(PT_all$Well_Label))
        PT_all$Well_Channel<-paste0(PT_all$Well_Label,"_",PT_all$Channel_Label)
        PT_all$Well_Channel<-factor(PT_all$Well_Channel, levels=unique(PT_all$Well_Channel))
        
        PT_all$Timestamp_ms<-PT_all$Timestamp*1000
        Spike_count<-data.frame(table(PT_all$Well_Channel))
        colnames(Spike_count)<-c("Well_Channel_all","Spike_count")
        #spikes will be the input for mi_find_bursts
        list_PT_all<-split(PT_all, f=PT_all$Well_Channel)
        channels<-names(list_PT_all)
        spikes<-PT_all[, c("Well_Channel","Timestamp")]
        spikes<-split(spikes, f=spikes$Well_Channel)
        spikes<-lapply(spikes,'[[',"Timestamp")
        well<-as.character(levels(as.factor(PT_all$Well_Label)))
        electrodes_per_well<-length(levels(factor(PT_all$Channel_Label)))
        active_wells<-names(spikes)
        active_wells<-strsplit(active_wells, "_")
        active_wells <- sapply(active_wells, function(x) x[1])
        active_wells<-unique(active_wells)
        inactive_well<-unique(Well_annotations$Well_Label)
        inactive_well<-setdiff(inactive_well, active_wells)
        cell_line_well<-"well_cell_lines.csv"
        numb_active_wells<-length(active_wells)
        
        if (file.exists(cell_line_well)) {
            cat("There is a csv with samples")
            cell_line<-read.csv("well_cell_lines.csv", fileEncoding="UTF-8-BOM") 
            cell_line <- subset(cell_line, !(Well %in% inactive_well))
            #cell_line <- subset(cell_line, cell_line != 'Exclude')
            cell_line$Well<-factor(cell_line$Well,levels=unique(cell_line$Well))
            cell_line <- cell_line[order(cell_line$Well), ]
            cell_line <- cell_line$cell_line
            } else {
            cat("Cell line id should be included with path")
            cell_line_name<-basename(dirname(PT_all_dir))
            cell_line<-rep(cell_line_name,numb_active_wells)
            } 
        
        #Create an output folder to store analysis results and indicate whether hfNB detection is ON or OFF 
        if (abs(hfNB_max_duration) < 1e-6 && abs(hfNB_max_time_interval) <1e-6) {
            hfNB_detect = "OFF"
        } else {
            hfNB_detect = "ON"
        }
        output_folder = paste0("Results_analysis_s",sigma_selected,"prop",min_electrodes_proportion,"_freqNB",
                          Min_freq_of_NB,"_in_",min_electrodes_proportion_in_Min_freq_of_NB, "_hfNBdetection", hfNB_detect)
        
        full_output_path <- file.path(PT_all_dir, output_folder)                
        dir.create(full_output_path, showWarnings = FALSE, recursive = TRUE)
        setwd(full_output_path)            
                               
        mi_par<-list("beg_isi"=Max_interval_to_start_bursts,"end_isi"=Max_interval_to_end_a_burst, 
        "min_ibi"=Min_interval_between_bursts,"min_durn"=Min_duration_of_burst,
        "min_spikes"=Min_number_of_spikes_in_burst )
        my_spikes<-data.frame(table(PT_all$Well_Channel))
        colnames(my_spikes)<-c("Well_Channel", "Spike_count")
        nspikes <- setNames(as.integer(my_spikes$Spike_count),as.character(my_spikes$Well_Channel))
        NCells<-length(spikes)
        meanfiringrate<-my_spikes$Spike_count/recording_time_sec
        meanfiringrate <- setNames(as.integer(meanfiringrate),as.character(my_spikes$Well_Channel))
        str(meanfiringrate)
        well<-as.character(levels(as.factor(PT_all$Well_Label)))

        names(cell_line)<-well
        parameters<-list("mi_par"=mi_par,"burst_type"="mi", "local_region_min_nae"=local_region_min_nae,
                "min_electrodes"=min_electrodes, "sigmas"=sigmas)

        s <- list("channels"=channels, "spikes"=spikes, "nspikes"=nspikes, "NCells"=NCells, "meanfiringrate"=meanfiringrate,
          "cell_line"=cell_line,"well"=well,  "file"=file,"parameters"=parameters)
        #use the mi_find_bursts function defined above to identify bursts
        s$allb <- lapply(s$spikes, mi_find_bursts, s$parameters$mi_par ) 
        #electrode data containing burst information is stored in electrode_burst_summary
        electrode_burst_summary<-calc_burst_summary(s, bursty_threshold = 1)#bursty_threshold = 1
        electrode_burst_summary$Well_Label<-electrode_burst_summary$channels
        electrode_burst_summary$burst_frequency<-electrode_burst_summary$nbursts/recording_time_sec
        tail(electrode_burst_summary)
        write.csv(electrode_burst_summary, "electrode_burst_summary.csv") #output file with bursts summary data per electrode
            
        #similar to .nb_extract_features of meaRtools
        min_electrodes=parameters$min_electrodes
        local_region_min_nae=parameters$local_region_min_nae
        duration =0 
        bin = 25 #bin: 25; #2ms, this is a parameter based on our recordin
        
        skip = 1
        df.spikes <- .nb_prepare(s)
        wells<-strsplit(colnames(df.spikes), "_")
        wells <- unique(sapply(wells, function(x) x[1]))                     
        output <- list()
        hf_output <- list()
        sbegin <- floor(min(df.spikes[df.spikes > 0] / skip)) * skip + skip
        send <- floor(max(df.spikes[df.spikes > 0] / skip)) * skip
        timespan <- send - sbegin

        bin_time <- bin / sf
        nb_times <- list()
        hf_nb_times <- list()
        length(nb_times) <- length(wells)
        names(nb_times) <- wells
        
        
        length(hf_nb_times) <- length(wells)
        names(hf_nb_times) <- wells
        
        
        cat(paste0("calculating network bursts for recording ",
             basename(s$file), "\n"))
        for (well.index in 1: length(wells)) {
            well <- wells[well.index]
            temp_return <- .nb_select_and_bin(df.spikes, well, sbegin, send, bin)
            offset2 <- temp_return[[2]]
            well_data <- temp_return[[1]]
            f <- list()
            for (i  in 1: length(sigmas)) {
              f[[i]] <- .nb_gaussian_filter(sigmas[i], well_data, min_electrodes)

                }
            names(f) <- sigmas

            stat0 <- .nb_get_spike_stats(well_data, timespan)
            res1 <- .nb_get_burst_stats_intensities(well_data,
            f, timespan, bin_time, min_electrodes,min_electrodes_proportion_in_Min_freq_of_NB_numb,
            sbegin, send, offset2,bin_size=0.002,hfNB_max_duration,hfNB_max_time_interval)
            stat1 <- res1[[1]]
            nb_times[[well.index]] <- res1[[2]]
            
            hf_stat1 <- res1[[3]] #Hf_NB
            hf_nb_times[[well.index]] <- res1[[4]] #hf_NB times

            output[[well.index]] <- list()
            output[[well.index]]$stat0 <- stat0
            output[[well.index]]$stat1 <- stat1  
            output[[well.index]]$well <- well
            
            hf_output[[well.index]] <- list()
            hf_output[[well.index]]$stat0 <- stat0
            hf_output[[well.index]]$hf_stat1 <- hf_stat1  
            hf_output[[well.index]]$well <- well
            
            
            
  }
        extract_features<-list(data = output,
        div = unlist(strsplit(unlist(
        strsplit(basename(s$file), "_"))[4], "[.]"))[1],
        nb_times = nb_times)
        
        hf_extract_features<-list(data = hf_output,
        div = unlist(strsplit(unlist(
        strsplit(basename(s$file), "_"))[4], "[.]"))[1],
        hf_nb_times = hf_nb_times)
        
        wells <- character()
        divs <- character()
        phenotypes <- character()
        data_all <- numeric()
        
        hf_wells <- character()
        hf_divs <- character()
        hf_phenotypes <- character()
        hf_data_all <- numeric()

        w_fun <- function(x) {
            x$well}

        current <- extract_features
        current_wells <- sapply(current$data, w_fun)
        divs <- DIV_range
        wells <- c(wells, current_wells)
        phenotypes <- cell_line
        

        hf_current <- hf_extract_features
        hf_current_wells <- sapply(hf_current$data, w_fun)
        hf_divs <- DIV_range
        hf_wells <- c(hf_wells, hf_current_wells)
        hf_phenotypes <- cell_line
        
        
        
        data <- matrix(0, length(current_wells),
                   length(c(as.vector(current$data[[1]]$stat1),
                            as.vector(current$data[[1]]$stat0))))
        
        hf_data <- matrix(0, length(hf_current_wells),
                   length(c(as.vector(hf_current$data[[1]]$hf_stat1),
                            as.vector(hf_current$data[[1]]$stat0))))
        
        
        for (j in 1:length(current_wells)) {
              data[j, ] <- c(as.vector(current$data[[j]]$stat1),
                     as.vector(current$data[[j]]$stat0))
                    }
        for (j in 1:length(hf_current_wells)) {
              hf_data[j, ] <- c(as.vector(hf_current$data[[j]]$hf_stat1),
                     as.vector(hf_current$data[[j]]$stat0))
                    }

        data_all <- rbind(data_all, data)
        
        hf_data_all <- rbind(hf_data_all, hf_data)


        feature_names <- character()
        hf_feature_names <- character()
        
        for (i in 1:length(sigmas)) {
            feature_names <- c(feature_names,
            paste(rownames(current$data[[1]]$stat1), sigmas[i], sep = "_"))
              }
        feature_names <- c(feature_names, colnames(current$data[[1]]$stat0))
        
        for (i in 1:length(sigmas)) {
            hf_feature_names <- c(hf_feature_names,
            paste(rownames(hf_current$data[[1]]$hf_stat1), sigmas[i], sep = "_"))
              }
        hf_feature_names <- c(hf_feature_names, colnames(hf_current$data[[1]]$stat0))

        df <- data.frame(divs, wells, phenotypes, data_all)
        
        hf_df <- data.frame(hf_divs, hf_wells, hf_phenotypes, hf_data_all)

        names(df)[4:dim(df)[2]] <- feature_names
        
        names(hf_df)[4:dim(hf_df)[2]] <- hf_feature_names

        write.csv(df, "NB_results.csv")

        write.csv(hf_df, "hf_NB_results.csv") 

        electrode_burst_summary$Well_Label<-sub("_.+", "", electrode_burst_summary$channels)

                     
        #well_summary will contain well averaged data
        well_summary<-as.data.frame(electrode_burst_summary %>% group_by(Well_Label) %>% summarise(sum(spikes)))
        colnames(well_summary)<-c("Well_Label", "Number_of_spikes")

        Mean_Firing_Rate_Hz<- electrode_burst_summary %>% group_by(Well_Label) %>% 
          summarise(Mean_Firing_Rate_Hz=mean(mean_freq,na.rm=TRUE), .groups = 'drop') %>%
          as.data.frame()
        Mean_Firing_Rate_Hz<-Mean_Firing_Rate_Hz$Mean_Firing_Rate_Hz
        well_summary$Mean_Firing_Rate_Hz<-Mean_Firing_Rate_Hz

        Number_of_Bursts<-electrode_burst_summary %>% group_by(Well_Label) %>% 
          summarise(Number_of_Bursts=sum(nbursts), .groups = 'drop')
        Number_of_Bursts<-Number_of_Bursts$Number_of_Bursts
        well_summary$Number_of_Bursts<-Number_of_Bursts

        Number_of_Bursting_Electrodes<-electrode_burst_summary %>% group_by(Well_Label) %>% 
          summarise(Number_of_Bursting_Electrodes=sum(bursty), .groups = 'drop')
        Number_of_Bursting_Electrodes<-Number_of_Bursting_Electrodes$Number_of_Bursting_Electrodes
        well_summary$Number_of_Bursting_Electrodes<-Number_of_Bursting_Electrodes

        Burst_Duration_Avg_sec<-electrode_burst_summary %>% group_by(Well_Label) %>% 
          summarise(Burst_Duration_Avg_sec=mean(mean_dur,na.rm=TRUE), .groups = 'drop')
        Burst_Duration_Avg_sec<-Burst_Duration_Avg_sec$Burst_Duration_Avg_sec
        well_summary$Burst_Duration_Avg_sec<-Burst_Duration_Avg_sec

        Burst_Duration_Std_sec<-electrode_burst_summary %>% group_by(Well_Label) %>% 
          summarise(Burst_Duration_Std_sec=sd(mean_dur,na.rm=TRUE), .groups = 'drop')
        Burst_Duration_Std_sec<-Burst_Duration_Std_sec$Burst_Duration_Std_sec
        well_summary$Burst_Duration_Std_sec<-Burst_Duration_Std_sec

        Number_of_Spikes_per_Burst_Avg<-electrode_burst_summary %>% group_by(Well_Label) %>% 
          summarise(Number_of_Spikes_per_Burst_Avg=mean(mean_spikes,na.rm=TRUE), .groups = 'drop')
        Number_of_Spikes_per_Burst_Avg<-Number_of_Spikes_per_Burst_Avg$Number_of_Spikes_per_Burst_Avg
        well_summary$Number_of_Spikes_per_Burst_Avg<-Number_of_Spikes_per_Burst_Avg

        Number_of_Spikes_per_Burst_Std<-electrode_burst_summary %>% group_by(Well_Label) %>% 
          summarise(Number_of_Spikes_per_Burst_Std=sd(mean_spikes,na.rm=TRUE), .groups = 'drop')
        Number_of_Spikes_per_Burst_Std<-Number_of_Spikes_per_Burst_Std$Number_of_Spikes_per_Burst_Std
        well_summary$Number_of_Spikes_per_Burst_Std<-Number_of_Spikes_per_Burst_Std

        Mean_ISI_within_Burst_Avg_sec<-electrode_burst_summary %>% group_by(Well_Label) %>% 
          summarise(Mean_ISI_within_Burst_Avg_sec=mean(mean_isis,na.rm=TRUE), .groups = 'drop')
        Mean_ISI_within_Burst_Avg_sec<-Mean_ISI_within_Burst_Avg_sec$Mean_ISI_within_Burst_Avg_sec
        well_summary$Mean_ISI_within_Burst_Avg_sec<-Mean_ISI_within_Burst_Avg_sec

        Mean_ISI_within_Burst_Std_sec<-electrode_burst_summary %>% group_by(Well_Label) %>% 
          summarise(Mean_ISI_within_Burst_Std_sec=sd(mean_isis,na.rm=TRUE), .groups = 'drop')
        Mean_ISI_within_Burst_Std_sec<-Mean_ISI_within_Burst_Std_sec$Mean_ISI_within_Burst_Std_sec
        well_summary$Mean_ISI_within_Burst_Std_sec<-Mean_ISI_within_Burst_Std_sec

        Inter_Burst_Interval_Avg_sec<-electrode_burst_summary %>% group_by(Well_Label) %>% 
          summarise(Inter_Burst_Interval_Avg_sec=mean(mean_ibis,na.rm=TRUE), .groups = 'drop')
        Inter_Burst_Interval_Avg_sec<-Inter_Burst_Interval_Avg_sec$Inter_Burst_Interval_Avg_sec
        well_summary$Inter_Burst_Interval_Avg_sec<-Inter_Burst_Interval_Avg_sec

        Inter_Burst_Interval_Std_sec<-electrode_burst_summary %>% group_by(Well_Label) %>% 
          summarise(Inter_Burst_Interval_Std_sec=sd(mean_ibis,na.rm=TRUE), .groups = 'drop')
        Inter_Burst_Interval_Std_sec<-Inter_Burst_Interval_Std_sec$Inter_Burst_Interval_Std_sec
        well_summary$Inter_Burst_Interval_Std_sec<-Inter_Burst_Interval_Std_sec

        IBI_Coefficient_of_Variation_Avg<-electrode_burst_summary %>% group_by(Well_Label) %>% 
          summarise(IBI_Coefficient_of_Variation_Avg=mean(cv_ibis,na.rm=TRUE), .groups = 'drop')
        IBI_Coefficient_of_Variation_Avg<-IBI_Coefficient_of_Variation_Avg$IBI_Coefficient_of_Variation_Avg
        well_summary$IBI_Coefficient_of_Variation_Avg<-IBI_Coefficient_of_Variation_Avg

        IBI_Coefficient_of_Variation_Std<-electrode_burst_summary %>% group_by(Well_Label) %>% 
          summarise(IBI_Coefficient_of_Variation_Std=sd(cv_ibis,na.rm=TRUE), .groups = 'drop')
        IBI_Coefficient_of_Variation_Std<-IBI_Coefficient_of_Variation_Std$IBI_Coefficient_of_Variation_Std
        well_summary$IBI_Coefficient_of_Variation_Std<-IBI_Coefficient_of_Variation_Std

        Burst_Frequency_Avg_Hz<-electrode_burst_summary %>% group_by(Well_Label) %>% 
          summarise(Burst_Frequency_Avg_Hz=mean(burst_frequency,na.rm=TRUE), .groups = 'drop')
        Burst_Frequency_Avg_Hz<-Burst_Frequency_Avg_Hz$Burst_Frequency_Avg_Hz
        well_summary$Burst_Frequency_Avg_Hz<-Burst_Frequency_Avg_Hz

        Burst_Frequency_Std_Hz<-electrode_burst_summary %>% group_by(Well_Label) %>% 
          summarise(Burst_Frequency_Std_Hz=sd(burst_frequency,na.rm=TRUE), .groups = 'drop')
        Burst_Frequency_Std_Hz<-Burst_Frequency_Std_Hz$Burst_Frequency_Std_Hz
        well_summary$Burst_Frequency_Std_Hz<-Burst_Frequency_Std_Hz

        Burst_Percentage_Avg<-electrode_burst_summary %>% group_by(Well_Label) %>% 
          summarise(Burst_Percentage_Avg=mean(per_spikes_in_burst,na.rm=TRUE), .groups = 'drop')
        Burst_Percentage_Avg<-Burst_Percentage_Avg$Burst_Percentage_Avg
        well_summary$Burst_Percentage_Avg<-Burst_Percentage_Avg

        Burst_Percentage_Std<-electrode_burst_summary %>% group_by(Well_Label) %>% 
          summarise(Burst_Percentage_Std=sd(per_spikes_in_burst,na.rm=TRUE), .groups = 'drop')
        Burst_Percentage_Std<-Burst_Percentage_Std$Burst_Percentage_Std
        well_summary$Burst_Percentage_Std<-Burst_Percentage_Std

        Random_Spikes_Percentage_Avg<-electrode_burst_summary %>% group_by(Well_Label) %>% 
          summarise(Random_Spikes_Percentage_Avg=mean(per_spikes_out_burst,na.rm=TRUE), .groups = 'drop')
        Random_Spikes_Percentage_Avg<-Random_Spikes_Percentage_Avg$Random_Spikes_Percentage_Avg
        well_summary$Random_Spikes_Percentage_Avg<-Random_Spikes_Percentage_Avg
        
        dfsigma_selected<-df[,c("divs","wells","phenotypes")] #df cointains NB results
        dfsigma_selected<-cbind(dfsigma_selected,df[,26:36])#columns cointaining data of window=100
        
        hf_dfsigma_selected<-hf_df[,c("hf_divs","hf_wells","hf_phenotypes")] #df cointains NB results
        hf_dfsigma_selected<-cbind(hf_dfsigma_selected,hf_df[,26:36])#columns cointaining data of window=100
        #include NB metrics  to well_summary
        well_summary$Phenotype<-dfsigma_selected$phenotypes[match(well_summary$Well_Label, dfsigma_selected$wells)]
        well_summary$Number_of_Network_Bursts<-dfsigma_selected[[paste0("total_number_of_NBs_",sigma_selected)]][match(well_summary$Well_Label, dfsigma_selected$wells)]
        well_summary$Number_of_hf_Network_Bursts<-hf_dfsigma_selected[[paste0("total_number_of_hf_NBs_",sigma_selected)]][match(well_summary$Well_Label, hf_dfsigma_selected$hf_wells)]
        well_summary$Network_Burst_Frequency<-well_summary$Number_of_Network_Bursts/recording_time_sec #in Hz
        well_summary$hf_Network_Burst_Frequency<-well_summary$Number_of_hf_Network_Bursts/recording_time_sec #in Hz
        well_summary$Spike_Frequency_per_Electrode<-dfsigma_selected[[paste0("spike_intensity_",sigma_selected)]][match(well_summary$Well_Label, dfsigma_selected$wells)]
        well_summary$hf_Spike_Frequency_per_Electrode<-hf_dfsigma_selected[[paste0("spike_intensity_",sigma_selected)]][match(well_summary$Well_Label, hf_dfsigma_selected$hf_wells)]
        well_summary$total_spikes_in_all_NBs_d_nNB<-dfsigma_selected[[paste0("total_spikes_in_all_NBs_d_nNB_",sigma_selected)]][match(well_summary$Well_Label, dfsigma_selected$wells)]
        well_summary$total_spikes_in_all_hf_NBs_d_nNB<-hf_dfsigma_selected[[paste0("total_spikes_in_all_hf_NBs_d_nNB_",sigma_selected)]][match(well_summary$Well_Label, hf_dfsigma_selected$hf_wells)]
        well_summary$Network_Burst_Percentage<-dfsigma_selected[[paste0("percent_of_spikes_in_NBs_",sigma_selected)]][match(well_summary$Well_Label, dfsigma_selected$wells)]
        well_summary$hf_Network_Burst_Percentage<-hf_dfsigma_selected[[paste0("percent_of_spikes_in_hf_NBs_",sigma_selected)]][match(well_summary$Well_Label, hf_dfsigma_selected$hf_wells)]

        # move age to first column
        well_summary = well_summary[ , c("Phenotype",
                                names(well_summary)[names(well_summary) != "Phenotype"])]


        nb_timessigma_selected<-list()
        for (i in names(nb_times)){
            df<-nb_times[[i]][[3]]
            df<-df[,c("start_t", "end_t","mean_firing_rate_in_nb")] 
            df$duration<-df$end_t - df$start_t

            nb_timessigma_selected[[i]]<-df
        }
        nb_timessigma_selected_avg<-list()
        for (i in names(nb_timessigma_selected)){
            df<-nb_timessigma_selected[[i]]
            mean<-mean(df$duration)
            sd<-sd(df$duration)

        
            mean_mean_firing_rate_in_nb<-suppressWarnings(mean(df$mean_firing_rate_in_nb))
            std_mean_firing_rate_in_nb<-sd(df$mean_firing_rate_in_nb)                 
            
            df<-data.frame("Avg"=mean, "Std"=sd,"Avg_mfr"=mean_mean_firing_rate_in_nb, "Std_mfr"=std_mean_firing_rate_in_nb ) #New
            nb_timessigma_selected_avg[[i]]<-df
        }
        nb_timessigma_selected_avg<-do.call("rbind", nb_timessigma_selected_avg)
        colnames(nb_timessigma_selected_avg)<-c("Network_Burst_Duration_Avg_sec", "Network_Burst_Duration_Std_sec",
                                               "Firing_Rate_in_NB_Avg", "Firing_Rate_in_NB_std")#end new
        nb_timessigma_selected_avg$Well_Label<-rownames(nb_timessigma_selected_avg)
        rownames(nb_timessigma_selected_avg)<-NULL
        well_summary_test<-well_summary
        well_summary_test<-merge(x=well_summary_test,y=nb_timessigma_selected_avg,by="Well_Label",all.x=TRUE)
        
        hf_nb_timessigma_selected<-list()
        for (i in names(hf_nb_times)){
            df<-hf_nb_times[[i]][[3]]
            df<-df[,c("hf_start_t", "hf_end_t","mean_firing_rate_in_hf_nb")]
            df$duration<-df$hf_end_t - df$hf_start_t
            hf_nb_timessigma_selected[[i]]<-df
        }
        hf_nb_timessigma_selected_avg<-list()
        for (i in names(hf_nb_timessigma_selected)){
            df<-hf_nb_timessigma_selected[[i]]
            mean<-mean(df$duration)
            sd<-sd(df$duration)

        
            suppressWarnings(mean_mean_firing_rate_in_hf_nb<-mean(df$mean_firing_rate_in_hf_nb))
            std_mean_firing_rate_in_hf_nb<-sd(df$mean_firing_rate_in_hf_nb)
            
            
            df<-data.frame("Avg"=mean, "Std"=sd,"Avg_mfr"=mean_mean_firing_rate_in_hf_nb, "Std_mfr"=std_mean_firing_rate_in_hf_nb)#New
            hf_nb_timessigma_selected_avg[[i]]<-df
        }
        hf_nb_timessigma_selected_avg<-do.call("rbind", hf_nb_timessigma_selected_avg)
        colnames(hf_nb_timessigma_selected_avg)<-c("hf_Network_Burst_Duration_Avg_sec", "hf_Network_Burst_Duration_Std_sec",
                                               "hf_Firing_Rate_in_NB_Avg", "hf_Firing_Rate_in_NB_std")#end new
        hf_nb_timessigma_selected_avg$Well_Label<-rownames(hf_nb_timessigma_selected_avg)
        rownames(hf_nb_timessigma_selected_avg)<-NULL
        
        well_summary_test<-merge(x=well_summary_test,y=hf_nb_timessigma_selected_avg,by="Well_Label",all.x=TRUE)
        

        nb_timessigma_selected_NBI<-nb_timessigma_selected
        nb_timessigma_selected_NBI_list<-list()
        well_NB<-names(nb_timessigma_selected_NBI)

        if(length(well_NB)>0){

            for (i in well_NB){    
                 df_nb_times_NBI<-nb_timessigma_selected_NBI[[i]]
                if(is.null(dim(df_nb_times_NBI)[1])== TRUE || dim(df_nb_times_NBI)[1]==0){
                    df_nb_times_NBI<-data.frame("start_t"=NA,"end_t"=NA,
                                               "duration"=NA,"NB_time_interval_back"=NA,
                                               "NB_time_interval_forward"=NA,"mean_firing_rate_in_nb"=NA )
                    
                }
                 df_nb_times_NBI$NB_time_interval_back<-NA
                 df_nb_times_NBI$NB_time_interval_forward<-NA
                 df_nb_times_NBI$Well_Label<-i
                 numb_NB<-nrow(df_nb_times_NBI)
                if(is.null(dim(df_nb_times_NBI)[1])== FALSE && nrow(df_nb_times_NBI) > 0){
                 
                     if(numb_NB>1){        
                        for (current_NB in 2:numb_NB){
                        df_nb_times_NBI[current_NB,"NB_time_interval_back"]<-df_nb_times_NBI[current_NB,"start_t"]-df_nb_times_NBI[current_NB-1,"end_t"]
                        }
                        for (current_NB in 1:numb_NB-1){
                        df_nb_times_NBI[current_NB,"NB_time_interval_forward"]<-df_nb_times_NBI[current_NB+1,"start_t"]-df_nb_times_NBI[current_NB,"end_t"]
                        }
                       }

                 

             }
                nb_timessigma_selected_NBI_list[[i]]<-df_nb_times_NBI
           }
            
        
          

        }
        nb_timessigma_selected_NBI_list_2<-nb_timessigma_selected_NBI_list
        nb_timessigma_selected_NBI_list<-do.call(rbind,nb_timessigma_selected_NBI_list)
            nb_timessigma_selected_NBI_list$Phenotype<-dfsigma_selected$phenotypes[match(nb_timessigma_selected_NBI_list$Well_Label, dfsigma_selected$wells)]
            nb_timessigma_selected_NBI_list$batch<-batch
            nb_timessigma_selected_NBI_list$cell_line_batch<-paste0(nb_timessigma_selected_NBI_list$Phenotype,nb_timessigma_selected_NBI_list$batch)
            nb_timessigma_selected_NBI_list$machine<-"axion"
            nb_timessigma_selected_NBI_list$PT_all_path<-file
            nb_timessigma_selected_NBI_list$DIV_range<-DIV_range
        write.csv(nb_timessigma_selected_NBI_list,paste0("nb_times_s",sigma_selected,".csv"))
        
        nb_timessigma_selected_avg_NBI<-list()
        for (i in names(nb_timessigma_selected_NBI_list_2)){
            df<-nb_timessigma_selected_NBI_list_2[[i]]
            mean<-mean(df$NB_time_interval_forward,na.rm=TRUE)
            sd<-sd(df$NB_time_interval_forward,na.rm=TRUE)
            df<-data.frame("Avg_NBI"=mean, "Std_NBI"=sd)
            nb_timessigma_selected_avg_NBI[[i]]<-df
        }
        nb_timessigma_selected_avg_NBI<-do.call("rbind", nb_timessigma_selected_avg_NBI)
        colnames(nb_timessigma_selected_avg_NBI)<-c("NBI_Avg_sec", "NBI_Avg_Std_sec")
        nb_timessigma_selected_avg_NBI$Well_Label<-rownames(nb_timessigma_selected_avg_NBI)
        rownames(nb_timessigma_selected_avg_NBI)<-NULL
        
        well_summary_test<-merge(x=well_summary_test,y=nb_timessigma_selected_avg_NBI,by="Well_Label",all.x=TRUE)
        
        
       hf_nb_timessigma_selected_NBI<-hf_nb_timessigma_selected
        hf_nb_timessigma_selected_NBI_list<-list()
        well_NB<-names(hf_nb_timessigma_selected_NBI)

        if(length(well_NB)>0){

            for (i in well_NB){    
                 df_nb_times_NBI<-hf_nb_timessigma_selected_NBI[[i]]
                if(is.null(dim(df_nb_times_NBI)[1])== TRUE || dim(df_nb_times_NBI)[1]==0){
                    df_nb_times_NBI<-data.frame("hf_start_t"=NA,"hf_end_t"=NA,
                                               "duration"=NA,"NB_time_interval_back"=NA,
                                               "NB_time_interval_forward"=NA,"mean_firing_rate_in_hf_nb"=NA )
                    
                }
                
                 df_nb_times_NBI$NB_time_interval_back<-NA
                 df_nb_times_NBI$NB_time_interval_forward<-NA
                 df_nb_times_NBI$Well_Label<-i
                 numb_NB<-nrow(df_nb_times_NBI)
                if(is.null(dim(df_nb_times_NBI)[1])== FALSE && nrow(df_nb_times_NBI) > 0){

                 
                     if(numb_NB>1){        
                        for (current_NB in 2:numb_NB){
                        df_nb_times_NBI[current_NB,"NB_time_interval_back"]<-df_nb_times_NBI[current_NB,"hf_start_t"]-df_nb_times_NBI[current_NB-1,"hf_end_t"]
                        }
                        for (current_NB in 1:numb_NB-1){
                        df_nb_times_NBI[current_NB,"NB_time_interval_forward"]<-df_nb_times_NBI[current_NB+1,"hf_start_t"]-df_nb_times_NBI[current_NB,"hf_end_t"]
                        }
                       }

                 

             }
                hf_nb_timessigma_selected_NBI_list[[i]]<-df_nb_times_NBI

           }
            

        }
        

        
        hf_nb_timessigma_selected_NBI_list_2<-hf_nb_timessigma_selected_NBI_list
        hf_nb_timessigma_selected_NBI_list<-do.call(rbind,hf_nb_timessigma_selected_NBI_list)
        hf_nb_timessigma_selected_NBI_list$Phenotype<-hf_dfsigma_selected$phenotypes[match(hf_nb_timessigma_selected_NBI_list$Well_Label, hf_dfsigma_selected$wells)]
        hf_nb_timessigma_selected_NBI_list$batch<-batch
        hf_nb_timessigma_selected_NBI_list$cell_line_batch<-paste0(hf_nb_timessigma_selected_NBI_list$Phenotype,hf_nb_timessigma_selected_NBI_list$batch)
        hf_nb_timessigma_selected_NBI_list$machine<-"axion"
        hf_nb_timessigma_selected_NBI_list$PT_all_path<-file
        hf_nb_timessigma_selected_NBI_list$DIV_range<-DIV_range
        write.csv(hf_nb_timessigma_selected_NBI_list,paste0("hf_nb_times_s",sigma_selected,".csv"))
        
        hf_nb_timessigma_selected_avg_NBI<-list()
        for (i in names(hf_nb_timessigma_selected_NBI_list_2)){
            df<-hf_nb_timessigma_selected_NBI_list_2[[i]]
            mean<-mean(df$NB_time_interval_forward,na.rm=TRUE)
            sd<-sd(df$NB_time_interval_forward,na.rm=TRUE)
            df<-data.frame("hf_Avg_NBI"=mean, "hf_Std_NBI"=sd)
            hf_nb_timessigma_selected_avg_NBI[[i]]<-df
        }
        hf_nb_timessigma_selected_avg_NBI<-do.call("rbind", hf_nb_timessigma_selected_avg_NBI)
        colnames(hf_nb_timessigma_selected_avg_NBI)<-c("hf_NBI_Avg_sec", "hf_NBI_Avg_Std_sec")
        hf_nb_timessigma_selected_avg_NBI$Well_Label<-rownames(hf_nb_timessigma_selected_avg_NBI)
        rownames(hf_nb_timessigma_selected_avg_NBI)<-NULL

        
        
        well_summary_test$batch<-batch
        well_summary_test$cell_line_batch<-paste0(well_summary_test$Phenotype,well_summary_test$batch)
        well_summary_test$machine<-"axion"
        well_summary_test$PT_all_path<-file
        well_summary_test$DIV_range<-DIV_range
        
        write.csv(well_summary_test, paste0("Well_Summary_Parameters_hfNB_s",sigma_selected,".csv"))
        

        print(paste0("making raster plots for",PT_all_dir))
        dir_path <- paste0("raster_plots_sigma",sigma_selected,"_hf","prop",min_electrodes_proportion)
        if (!dir.exists(dir_path)) {
            dir.create(dir_path, showWarnings = FALSE)
        }
                               
        #RASTER PLOTS 
        Well_electrodes<-names(spikes)
        all_wells<-strsplit(Well_electrodes, "_")
        all_wells <- paste0(unique(sapply(all_wells, function(x) x[1])),"_")                              
        big_list_df_raster<-list()
    for (x in 1:length(all_wells)){
            to_keep<-grepl(all_wells[x], Well_electrodes, ignore.case ="True")
            Well_X<-Well_electrodes[to_keep]
            Well_X<-spikes[Well_X]
            list_df_raster<-list()



            active_electrodes<-names(Well_X)
            active_electrodes_n<-paste0("_",sub("^.*_([^_]+)$", "\\1", active_electrodes))
            
            
            active_electrodes<-data.frame("Electrode_Label_num"=active_electrodes_n)
            active_electrodes$Electrode_Label<-Electrode_annotations$Electrode_Label[match(active_electrodes$Electrode_Label_num,Electrode_annotations$Electrode_Label_num)]
            active_electrodes<-active_electrodes$Electrode_Label
            for (i in 1:length(active_electrodes)){
             electrode<-active_electrodes[i]
             well_electrode<-paste0(all_wells[x],substr(active_electrodes_n[i], start = 2 , stop = 3))
             ts=Well_X[[paste(well_electrode)]]
             df_test<-data.frame("spk"=ts, "electrode"=active_electrodes[i])
             list_df_raster[[i]]<-df_test
            }
          list_df_raster_bind<-do.call("rbind", list_df_raster)

            big_list_df_raster[[all_wells[x]]]<-list_df_raster_bind
        }

        for (i in names(big_list_df_raster)){
            
            
            well_no_line<-strsplit(i, "_")
            well_no_line <-sapply(well_no_line, function(x) x[1])
            
            

            all_start<-nb_timessigma_selected[[well_no_line]]$start_t
            all_end<-nb_timessigma_selected[[well_no_line]]$end_t
            al_NB<-c(all_start,all_end)
            
            
            hf_all_start<-hf_nb_timessigma_selected[[well_no_line]]$hf_start_t
            hf_all_end<-hf_nb_timessigma_selected[[well_no_line]]$hf_end_t
            hf_al_NB<-c(hf_all_start,hf_all_end)
            
            

            plot<-ggplot(big_list_df_raster[[i]],aes(x=spk,y=electrode))+
            geom_point(size=2, shape="|")+ theme_minimal()+
        geom_vline(xintercept = al_NB,linewidth = 0.5,
                      color=alpha("red",0.8)) +
            geom_vline(xintercept = hf_al_NB,linewidth = 0.4,
                      color=alpha("blue",0.8)) +
               scale_y_continuous(limits = c(1, electrodes_per_well), breaks = 1:electrodes_per_well)+ 
           # scale_x_continuous(limits = c(1, 100))+
            xlab("time [s]")
            
#              tiff(paste0("raster_plots_sigma",sigma_selected,"_hf/",i,"raster_NB_show.tiff"), res = 300, units = "in", width = 5, height =2.5)
#             print(plot)
#              dev.off()
            abs_path <- normalizePath(paste0(getwd(), "/raster_plots_sigma", sigma_selected, "_hfprop", min_electrodes_proportion))
            tiff(paste0(abs_path, "/", i, "raster.tiff"), res = 100, units = "in", width = 25, height =2.5)
            print(plot)
            dev.off()
            
#            pdf(file=paste0("raster_plots_sigma",sigma_selected,"_hfprop",min_electrodes_proportion,"/",i,"raster_NB_show.pdf"), width = 5, height = 2.5)
#             print(plot)
#             dev.off()
       # ggsave(paste0("raster_plots/",i,"raster_NB.svg"), plot=plot, width=10, height=3,dpi=300)

            }
        }
}else{print("no Axion_PT_all files")}
                                  
print("Analysis done!")

END OF CODE